# Análisis de rima con Rhyme Tagger

El código y documentación de la herramienta están disponibles en https://github.com/versotym/rhymetagger

Se trata de una herramienta multilingüe, las lenguas soportadas están en la documentación (español, francés, inglés, alemán, checo entre otras).

El uso es muy sencillo y la salida es más sencilla que Rantanplan. Proporciona el esquema rimático (p. ej. abab) pero no las rimas (contrariamente a Rantanplan). Otra diferencia con respecto a Rantanplan es que RhymeTagger no requiere una librería de análisis lingüistico.

Vemos primero paso a paso cómo usar la herramienta y después un ejemplo completo.

## Instalación

Se efectúa con el comando habitual para instalar módulos de Python:

In [1]:
!pip install rhymetagger

  Using cached rhymetagger-0.2.9.tar.gz (3.0 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 1.1 MB/s eta 0:00:00 MB/s eta 0:00:01
  Created wheel for rhymetagger: filename=rhymetagger-0.2.9-py3-none-any.whl size=3092942 sha256=868761d2837cf447871119c977389ae0f873dbde0e7d23d25486f83c7af0c310
  Stored in directory: /home/ruizfabo/.cache/pip/wheels/23/c6/53/5ba29373fa9f439fbd6a2991d38b2eca759f53023bc8e339de
Successfully built rhymetagger


## Uso

Importar la librería:

In [2]:
from rhymetagger import RhymeTagger

Instanciar un objeto de typo `RhymeTagger`. Este objeto se ocupa de hacer el análisis de rima.

In [3]:
rt = RhymeTagger()

Configurar el objeto para la lengua que se quiere analizar. Los códigos de lengua disponibles por defecto están en la [documentación](https://github.com/versotym/rhymetagger#pre-trained-models). Nosotros cargamos el modelo para español, entrenado sobre la base del corpus [DISCO](https://github.com/pruizf/disco).

Al configurar la lengua, se imprimen en pantalla algunas características del modelo. Se puede eliminar esta información con la opción `verbose=False`, es decir, cargando el modelo con `rt.load_model(model="es", verbose=False)` en este caso.

In [4]:
rt.load_model(model='es')

Model loaded with following settings:
  frequency_min: 3
           lang: es
       max_iter: 20
          ngram: 3
   ngram_length: 3
   prob_ipa_min: 0.9
 prob_ngram_min: 0.9
     same_words: False
   stanza_limit: True
         stress: True
       syll_max: 2
    t_score_min: 3.078
   vowel_length: True
         window: 5


El texto de nuestro poema es el ya utilizado para Rantanplan (poema de la autora filipina Evangelina Guerrero):

In [6]:
texto_poema = """Rosas sangrantes sobre el mar desflora
el sol que dice adioses en la tarde,
riman las aguas su canción sonora,
bajo nubes de fuego el poniente arde.

Vibran las cañas al chocar del viento,
formando extraña y triste sinfonía,
y la palmera altiva en vaivén lento
es una glauca nota de armonía.

Una barca se aleja lentamente,
una estela de luz, un vago canto,
sombras que pasan sobre el quieto mar;

Y las horas se van pausadamente,
mientras vierte la luz su último encanto
en un intenso, pálido llamear."""

RhymeTagger necesita como entrada una *lista* de versos, una lista de cadenas de texto, cada una de las cuales corresponde a un verso.

El código siguiente pasa de la cadena `texto_poema` a la lista de cadenas `texto_poema_lista`, gracias a la división del poema en versos usando el salto de línea `\n`, con `.split("\n")`. La expresión también elimina las líneas vacías, ya que estas no cumplen la condicion `if linea != ""`.

In [7]:
texto_poema_lista = [linea for linea in texto_poema.split("\n") if linea != ""]

In [8]:
texto_poema_lista

['Rosas sangrantes sobre el mar desflora',
 'el sol que dice adioses en la tarde,',
 'riman las aguas su canción sonora,',
 'bajo nubes de fuego el poniente arde.',
 'Vibran las cañas al chocar del viento,',
 'formando extraña y triste sinfonía,',
 'y la palmera altiva en vaivén lento',
 'es una glauca nota de armonía.',
 'Una barca se aleja lentamente,',
 'una estela de luz, un vago canto,',
 'sombras que pasan sobre el quieto mar;',
 'Y las horas se van pausadamente,',
 'mientras vierte la luz su último encanto',
 'en un intenso, pálido llamear.']

Para obtener las rimas, aplicamos el método `tag()` del objeto `RhymeTagger` (que está asignado a `rt`):

Los formatos de salida están descritos en la [documentación](https://github.com/versotym/rhymetagger#rhymetaggertagpoem-transcribedfalse-output_format1-kwargs), si bien el más práctico para nosotros es el formato `3`, que da la posición de la rima en el esquema rimático usando un índice numérico. Es decir, en vez de *a b a b c d c d*, por ejemplo, nos daría *1 2 1 2 3 4 3 4*. Hablando más estrictamente, nos da una lista con los números que corresponden a esas posiciones. Veremos después cómo traducir estos números a un esquema rimático expresado con letras.

In [19]:
rimas = rt.tag(texto_poema_lista, output_format=3)

In [20]:
rimas

[1, 2, 1, 2, 3, 4, 3, 4, 5, 6, 7, 5, 6, 7]

Para traducir el esquema a letras, podemos hacer lo siguiente:

In [13]:
from string import ascii_lowercase

In [30]:
rimas_letras = [ascii_lowercase[posicion_esquema-1]
                if posicion_esquema is not None
                else "-" for posicion_esquema in rimas]

In [15]:
rimas_letras

['a', 'b', 'a', 'b', 'c', 'd', 'c', 'd', 'e', 'f', 'g', 'e', 'f', 'g']

`ascii_lowercase` son las letras del alfabeto en inglés, dadas como una cadena de texto. Se puede acceder a cada carácter de una cadena de texto usando un índice numérico, empezando por 0. Es decir, el primer carácter de la cadena corresponde al índice 0, el segundo al índice 1, etc. Aprovecharemos esta propiedad de las cadenas en el lenguaje Python para traducir los índices numéricos devueltos por RhymeTagger a las letras correspondientes.

Para esto, debemos restar 1 al índice proporcionado por RhymeTagger, ya que RhymeTagger empieza a contar por 1, mientras que las posiciones de una cadena de Python se empiezan a describir a partir de 0, no 1. Esto es lo que hace la expresión `[ascii_lowercase[posicion_esquema-1] if posicion_esquema is not None else "-" for posicion_esquema in rimas]`.

Más exactamente, la expresión traduce el índice numérico a un índice alfabético, salvo en caso de que RhymeTagger no haya podido determinar la posición del verso en el esquema rimático. En tal caso, en vez de un número, RhymeTagger da un guion `-` como salida, que queda tal cual al pasar a índices alfabéticos.

Finalmente, podemos mostrar el número, texto, y posición en el esquema rimático de cada verso con el bucle siguiente. Hace uso de la instrucción `enumerate()`, que ya hemos visto en los notebooks sobre [escansión] con Rantanplan y [rima con Rantanplan]

In [29]:
for indice, verso in enumerate(texto_poema_lista):
    print(f"{indice+1}\t{verso}\t{rimas_letras[indice]}")

1	Rosas sangrantes sobre el mar desflora	a
2	el sol que dice adioses en la tarde,	b
3	riman las aguas su canción sonora,	a
4	bajo nubes de fuego el poniente arde.	b
5	Vibran las cañas al chocar del viento,	c
6	formando extraña y triste sinfonía,	d
7	y la palmera altiva en vaivén lento	c
8	es una glauca nota de armonía.	d
9	Una barca se aleja lentamente,	e
10	una estela de luz, un vago canto,	f
11	sombras que pasan sobre el quieto mar;	g
12	Y las horas se van pausadamente,	e
13	mientras vierte la luz su último encanto	f
14	en un intenso, pálido llamear.	g


Para mejorar la alineación del esquema rimático con respecto a los versos, se puede añadir `:<40` cuando se imprime en pantalla el texto de cada verso, en `{verso:<40}` dentro del `print()` final, con lo cual la column de los versos queda alineada a la izquierda y se rellenarán con espacios hasta alcanzar 40 caracteres aquellos versos que no llegan a 40 caracteres, de modo que el esquema rimático queda visualmente alineado.

In [26]:
for indice, verso in enumerate(texto_poema_lista):
    print(f"{indice+1}\t{verso:<40}\t{rimas_letras[indice]}")

1	Rosas sangrantes sobre el mar desflora  	a
2	el sol que dice adioses en la tarde,    	b
3	riman las aguas su canción sonora,      	a
4	bajo nubes de fuego el poniente arde.   	b
5	Vibran las cañas al chocar del viento,  	c
6	formando extraña y triste sinfonía,     	d
7	y la palmera altiva en vaivén lento     	c
8	es una glauca nota de armonía.          	d
9	Una barca se aleja lentamente,          	e
10	una estela de luz, un vago canto,       	f
11	sombras que pasan sobre el quieto mar;  	g
12	Y las horas se van pausadamente,        	e
13	mientras vierte la luz su último encanto	f
14	en un intenso, pálido llamear.          	g


## Ejemplo completo

In [32]:
from rhymetagger import RhymeTagger
from string import ascii_lowercase

rt = RhymeTagger()
rt.load_model(model='es', verbose=False)

texto_poema = """Rosas sangrantes sobre el mar desflora
el sol que dice adioses en la tarde,
riman las aguas su canción sonora,
bajo nubes de fuego el poniente arde.

Vibran las cañas al chocar del viento,
formando extraña y triste sinfonía,
y la palmera altiva en vaivén lento
es una glauca nota de armonía.

Una barca se aleja lentamente,
una estela de luz, un vago canto,
sombras que pasan sobre el quieto mar;

Y las horas se van pausadamente,
mientras vierte la luz su último encanto
en un intenso, pálido llamear."""

texto_poema_lista = [linea for linea in texto_poema.split("\n")
                     if linea != ""]

rimas = rt.tag(texto_poema_lista, output_format=3)

rimas_letras = [ascii_lowercase[posicion_esquema-1]
                if posicion_esquema is not None
                else "-" for posicion_esquema in rimas]

for indice, verso in enumerate(texto_poema_lista):
    print(f"{indice+1}\t{verso:<40}\t{rimas_letras[indice]}")

1	Rosas sangrantes sobre el mar desflora  	a
2	el sol que dice adioses en la tarde,    	b
3	riman las aguas su canción sonora,      	a
4	bajo nubes de fuego el poniente arde.   	b
5	Vibran las cañas al chocar del viento,  	c
6	formando extraña y triste sinfonía,     	d
7	y la palmera altiva en vaivén lento     	c
8	es una glauca nota de armonía.          	d
9	Una barca se aleja lentamente,          	e
10	una estela de luz, un vago canto,       	f
11	sombras que pasan sobre el quieto mar;  	g
12	Y las horas se van pausadamente,        	e
13	mientras vierte la luz su último encanto	f
14	en un intenso, pálido llamear.          	g
